# 第十一章 標準程式庫 (Part 2: 資料處理和網路通訊)

在 Part 2 中，我們將深入探討 Go 標準程式庫中用於資料處理和網路通訊的重要套件。這些套件是現代應用程式開發的基石，提供了處理 JSON 資料和 HTTP 通訊的強大功能。

我們將學習 `encoding/json` 套件的各種用法，以及 `net/http` 套件的客戶端和伺服器端程式設計。

## encoding/json

`encoding/json` 套件提供了 JSON 資料的編碼和解碼功能。JSON 是現代 web 應用程式中最常用的資料交換格式，Go 的 JSON 處理功能既強大又靈活。

### 核心概念：
- **Marshal**: 將 Go 資料結構編碼為 JSON
- **Unmarshal**: 將 JSON 解碼為 Go 資料結構
- **Encoder/Decoder**: 用於處理 JSON 串流
- **Struct Tags**: 用於控制 JSON 編碼行為

### 支援的資料類型：
- 基本型別：bool, 數字, 字串
- 陣列和切片
- Map (鍵必須是字串)
- 結構體
- 指標、介面

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
)

/* 基本 JSON 編碼 (Marshal) */
func basicMarshalExample() {
    fmt.Println("基本 JSON 編碼示例:")
    
    // 基本型別編碼
    name := "Alice"
    age := 30
    isActive := true
    
    nameJSON, _ := json.Marshal(name)
    ageJSON, _ := json.Marshal(age)
    activeJSON, _ := json.Marshal(isActive)
    
    fmt.Printf("字串: %s -> %s\n", name, nameJSON)
    fmt.Printf("整數: %d -> %s\n", age, ageJSON)
    fmt.Printf("布林: %t -> %s\n", isActive, activeJSON)
    
    // 陣列和切片編碼
    numbers := []int{1, 2, 3, 4, 5}
    numbersJSON, _ := json.Marshal(numbers)
    fmt.Printf("切片: %v -> %s\n", numbers, numbersJSON)
    
    // Map 編碼
    person := map[string]interface{}{
        "name":    "Bob",
        "age":     25,
        "active":  true,
        "hobbies": []string{"reading", "swimming"},
    }
    personJSON, _ := json.Marshal(person)
    fmt.Printf("Map: %v -> %s\n", person, personJSON)
    
    // 輸出:
    // 字串: Alice -> "Alice"
    // 整數: 30 -> 30
    // 布林: true -> true
    // 切片: [1 2 3 4 5] -> [1,2,3,4,5]
    // Map: map[active:true age:25 hobbies:[reading swimming] name:Bob] -> {"active":true,"age":25,"hobbies":["reading","swimming"],"name":"Bob"}
}

func main() {
    basicMarshalExample()
}

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
)

/* 基本 JSON 解碼 (Unmarshal) */
func basicUnmarshalExample() {
    fmt.Println("基本 JSON 解碼示例:")
    
    // 解碼基本型別
    var name string
    var age int
    var isActive bool
    
    json.Unmarshal([]byte(`"Charlie"`), &name)
    json.Unmarshal([]byte(`35`), &age)
    json.Unmarshal([]byte(`false`), &isActive)
    
    fmt.Printf("解碼字串: %s\n", name)
    fmt.Printf("解碼整數: %d\n", age)
    fmt.Printf("解碼布林: %t\n", isActive)
    
    // 解碼陣列
    var numbers []int
    json.Unmarshal([]byte(`[10, 20, 30, 40, 50]`), &numbers)
    fmt.Printf("解碼陣列: %v\n", numbers)
    
    // 解碼到 map
    var person map[string]interface{}
    personJSON := `{
        "name": "Diana",
        "age": 28,
        "active": true,
        "score": 95.5,
        "hobbies": ["painting", "dancing"]
    }`
    
    err := json.Unmarshal([]byte(personJSON), &person)
    if err != nil {
        fmt.Printf("解碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("解碼 Map:\n")
    for key, value := range person {
        fmt.Printf("  %s: %v (型別: %T)\n", key, value, value)
    }
    
    // 輸出:
    // 解碼字串: Charlie
    // 解碼整數: 35
    // 解碼布林: false
    // 解碼陣列: [10 20 30 40 50]
    // 解碼 Map:
    //   name: Diana (型別: string)
    //   age: 28 (型別: float64)
    //   active: true (型別: bool)
    //   score: 95.5 (型別: float64)
    //   hobbies: [painting dancing] (型別: []interface {})
}

func main() {
    basicUnmarshalExample()
}

### 使用 struct tag 來加入中繼資料

Struct tags 是 Go 中用於為結構體欄位添加中繼資料的機制。在 JSON 處理中，struct tags 用於控制 JSON 編碼和解碼的行為。

### 常用的 JSON struct tags：
- `json:"field_name"`: 指定 JSON 欄位名稱
- `json:"-"`: 忽略該欄位
- `json:"field_name,omitempty"`: 空值時省略
- `json:"field_name,string"`: 將數字轉為字串
- `json:",inline"` 或 `json:",embedded"`: 內嵌欄位

### 使用場景：
- API 回應格式控制
- 資料庫欄位對應
- 配置文件解析
- 不同命名慣例的轉換

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
)

/* 結構體和 struct tags 基本用法 */
type User struct {
    ID       int    `json:"id"`
    Name     string `json:"name"`
    Email    string `json:"email"`
    Age      int    `json:"age"`
    IsActive bool   `json:"is_active"`
    Password string `json:"-"` // 忽略此欄位
}

func structTagsExample() {
    fmt.Println("Struct Tags 示例:")
    
    user := User{
        ID:       1,
        Name:     "Alice Johnson",
        Email:    "alice@example.com",
        Age:      30,
        IsActive: true,
        Password: "secret123", // 這個不會出現在 JSON 中
    }
    
    // 編碼結構體
    userJSON, err := json.MarshalIndent(user, "", "  ")
    if err != nil {
        fmt.Printf("編碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("編碼結果:\n%s\n", userJSON)
    
    // 解碼 JSON 到結構體
    jsonStr := `{
        "id": 2,
        "name": "Bob Smith",
        "email": "bob@example.com",
        "age": 25,
        "is_active": false
    }`
    
    var newUser User
    err = json.Unmarshal([]byte(jsonStr), &newUser)
    if err != nil {
        fmt.Printf("解碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("\n解碼結果:\n")
    fmt.Printf("ID: %d\n", newUser.ID)
    fmt.Printf("Name: %s\n", newUser.Name)
    fmt.Printf("Email: %s\n", newUser.Email)
    fmt.Printf("Age: %d\n", newUser.Age)
    fmt.Printf("IsActive: %t\n", newUser.IsActive)
    fmt.Printf("Password: %s (應該為空)\n", newUser.Password)
}

func main() {
    structTagsExample()
}

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "time"
)

/* 進階 struct tags 用法 */
type Product struct {
    ID          int       `json:"id"`
    Name        string    `json:"name"`
    Price       float64   `json:"price"`
    Description string    `json:"description,omitempty"` // 空值時省略
    CategoryID  int       `json:"category_id,string"`    // 轉為字串
    InStock     *bool     `json:"in_stock,omitempty"`    // 指標，可為 nil
    CreatedAt   time.Time `json:"created_at"`
    Tags        []string  `json:"tags,omitempty"`        // 空切片時省略
    Internal    string    `json:"-"`                     // 完全忽略
}

func advancedStructTagsExample() {
    fmt.Println("進階 Struct Tags 示例:")
    
    // 建立產品實例
    inStock := true
    product1 := Product{
        ID:         1,
        Name:       "筆記型電腦",
        Price:      999.99,
        Description: "高性能筆記型電腦",
        CategoryID:  10,
        InStock:     &inStock,
        CreatedAt:   time.Now(),
        Tags:        []string{"electronics", "computer"},
        Internal:    "這是內部資訊",
    }
    
    // 編碼完整產品
    product1JSON, _ := json.MarshalIndent(product1, "", "  ")
    fmt.Printf("完整產品:\n%s\n\n", product1JSON)
    
    // 建立省略欄位的產品
    product2 := Product{
        ID:        2,
        Name:      "滑鼠",
        Price:     29.99,
        CategoryID: 11,
        CreatedAt: time.Now(),
        // Description, InStock, Tags 都會被省略
        Internal: "這也不會出現",
    }
    
    product2JSON, _ := json.MarshalIndent(product2, "", "  ")
    fmt.Printf("省略欄位的產品:\n%s\n\n", product2JSON)
    
    // 解碼測試
    jsonStr := `{
        "id": 3,
        "name": "鍵盤",
        "price": 79.99,
        "category_id": "12",
        "in_stock": true,
        "created_at": "2023-12-25T15:30:00Z",
        "tags": ["accessories", "input"]
    }`
    
    var product3 Product
    err := json.Unmarshal([]byte(jsonStr), &product3)
    if err != nil {
        fmt.Printf("解碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("解碼結果:\n")
    fmt.Printf("ID: %d\n", product3.ID)
    fmt.Printf("Name: %s\n", product3.Name)
    fmt.Printf("Price: %.2f\n", product3.Price)
    fmt.Printf("CategoryID: %d (從字串 '12' 轉換)\n", product3.CategoryID)
    if product3.InStock != nil {
        fmt.Printf("InStock: %t\n", *product3.InStock)
    }
    fmt.Printf("CreatedAt: %v\n", product3.CreatedAt)
    fmt.Printf("Tags: %v\n", product3.Tags)
    fmt.Printf("Internal: '%s' (應該為空)\n", product3.Internal)
}

func main() {
    advancedStructTagsExample()
}

### unmarshal 與 marshal

Marshal 和 Unmarshal 是 JSON 處理的核心函式。理解它們的工作原理和使用方法對於有效處理 JSON 資料至關重要。

### Marshal 過程：
1. 檢查型別是否實作了 `json.Marshaler` 介面
2. 檢查型別是否實作了 `encoding.TextMarshaler` 介面  
3. 使用預設的編碼規則

### Unmarshal 過程：
1. 檢查目標型別是否實作了 `json.Unmarshaler` 介面
2. 檢查目標型別是否實作了 `encoding.TextUnmarshaler` 介面
3. 使用預設的解碼規則

### 型別對應關係：
- JSON null → Go nil
- JSON boolean → Go bool
- JSON number → Go float64, int 等
- JSON string → Go string
- JSON array → Go slice, array
- JSON object → Go map, struct

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
)

/* 複雜資料結構的 Marshal 和 Unmarshal */
type Address struct {
    Street  string `json:"street"`
    City    string `json:"city"`
    Country string `json:"country"`
    ZipCode string `json:"zip_code,omitempty"`
}

type Person struct {
    ID        int                    `json:"id"`
    FirstName string                 `json:"first_name"`
    LastName  string                 `json:"last_name"`
    Age       int                    `json:"age"`
    Email     string                 `json:"email"`
    Addresses []Address              `json:"addresses"`
    Metadata  map[string]interface{} `json:"metadata,omitempty"`
}

func complexMarshalUnmarshalExample() {
    fmt.Println("複雜資料結構 Marshal/Unmarshal 示例:")
    
    // 建立複雜的資料結構
    person := Person{
        ID:        1,
        FirstName: "John",
        LastName:  "Doe",
        Age:       30,
        Email:     "john.doe@example.com",
        Addresses: []Address{
            {
                Street:  "123 Main St",
                City:    "New York",
                Country: "USA",
                ZipCode: "10001",
            },
            {
                Street:  "456 Oak Ave",
                City:    "Los Angeles",
                Country: "USA",
                ZipCode: "90210",
            },
        },
        Metadata: map[string]interface{}{
            "last_login":    "2023-12-25T10:30:00Z",
            "login_count":   42,
            "is_premium":    true,
            "preferences":   []string{"email", "sms"},
        },
    }
    
    // Marshal 為 JSON
    personJSON, err := json.MarshalIndent(person, "", "  ")
    if err != nil {
        fmt.Printf("Marshal 錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("Marshal 結果:\n%s\n\n", personJSON)
    
    // Unmarshal 回 Go 結構體
    var newPerson Person
    err = json.Unmarshal(personJSON, &newPerson)
    if err != nil {
        fmt.Printf("Unmarshal 錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("Unmarshal 結果:\n")
    fmt.Printf("姓名: %s %s\n", newPerson.FirstName, newPerson.LastName)
    fmt.Printf("年齡: %d\n", newPerson.Age)
    fmt.Printf("電子郵件: %s\n", newPerson.Email)
    
    fmt.Printf("地址:\n")
    for i, addr := range newPerson.Addresses {
        fmt.Printf("  地址 %d: %s, %s, %s %s\n", 
            i+1, addr.Street, addr.City, addr.Country, addr.ZipCode)
    }
    
    fmt.Printf("中繼資料:\n")
    for key, value := range newPerson.Metadata {
        fmt.Printf("  %s: %v (型別: %T)\n", key, value, value)
    }
}

func main() {
    complexMarshalUnmarshalExample()
}

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
)

/* 處理未知結構的 JSON 資料 */
func unknownStructureExample() {
    fmt.Println("處理未知結構 JSON 示例:")
    
    // 模擬從 API 收到的未知結構 JSON
    jsonData := `{
        "status": "success",
        "data": {
            "users": [
                {
                    "id": 1,
                    "name": "Alice",
                    "details": {
                        "age": 25,
                        "city": "New York"
                    }
                },
                {
                    "id": 2,
                    "name": "Bob",
                    "details": {
                        "age": 30,
                        "city": "Los Angeles"
                    }
                }
            ],
            "total": 2,
            "page": 1
        }
    }`
    
    // 使用 map[string]interface{} 來處理未知結構
    var result map[string]interface{}
    err := json.Unmarshal([]byte(jsonData), &result)
    if err != nil {
        fmt.Printf("解碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("狀態: %v\n", result["status"])
    
    // 安全地存取巢狀資料
    if data, ok := result["data"].(map[string]interface{}); ok {
        fmt.Printf("總數: %.0f\n", data["total"].(float64))
        fmt.Printf("頁面: %.0f\n", data["page"].(float64))
        
        if users, ok := data["users"].([]interface{}); ok {
            fmt.Printf("使用者:\n")
            for i, user := range users {
                if userMap, ok := user.(map[string]interface{}); ok {
                    fmt.Printf("  使用者 %d:\n", i+1)
                    fmt.Printf("    ID: %.0f\n", userMap["id"].(float64))
                    fmt.Printf("    名稱: %s\n", userMap["name"].(string))
                    
                    if details, ok := userMap["details"].(map[string]interface{}); ok {
                        fmt.Printf("    年齡: %.0f\n", details["age"].(float64))
                        fmt.Printf("    城市: %s\n", details["city"].(string))
                    }
                }
            }
        }
    }
}

func main() {
    unknownStructureExample()
}

### JSON、Reader 與 Writer

除了 `Marshal` 和 `Unmarshal` 函式外，`encoding/json` 套件還提供了 `Encoder` 和 `Decoder`，它們可以直接與 `io.Reader` 和 `io.Writer` 介面配合使用。

### Encoder 和 Decoder 的優勢：
- 可以直接從/向 `io.Reader`/`io.Writer` 讀寫
- 支援串流處理，不需要將整個 JSON 載入記憶體
- 適合處理大型 JSON 文件
- 可以設定各種編碼選項

### 使用場景：
- HTTP 回應處理
- 檔案 I/O
- 網路串流
- 管道處理

In [ ]:
package main

import (
    "bytes"
    "encoding/json"
    "fmt"
    "strings"
)

/* JSON Encoder 使用示例 */
type Book struct {
    ID     int    `json:"id"`
    Title  string `json:"title"`
    Author string `json:"author"`
    Price  float64 `json:"price"`
}

func jsonEncoderExample() {
    fmt.Println("JSON Encoder 示例:")
    
    books := []Book{
        {ID: 1, Title: "Go 程式設計", Author: "張三", Price: 45.99},
        {ID: 2, Title: "學習 JSON", Author: "李四", Price: 32.50},
        {ID: 3, Title: "Web 開發", Author: "王五", Price: 59.99},
    }
    
    // 使用 bytes.Buffer 作為 Writer
    var buffer bytes.Buffer
    encoder := json.NewEncoder(&buffer)
    
    // 設定縮排讓輸出更美觀
    encoder.SetIndent("", "  ")
    
    // 設定不轉義 HTML 字符
    encoder.SetEscapeHTML(false)
    
    fmt.Println("編碼多個物件:")
    for i, book := range books {
        err := encoder.Encode(book)
        if err != nil {
            fmt.Printf("編碼第 %d 本書錯誤: %v\n", i+1, err)
            continue
        }
        fmt.Printf("第 %d 本書編碼完成\n", i+1)
    }
    
    fmt.Printf("\n編碼結果:\n%s\n", buffer.String())
    
    // 編碼陣列
    var arrayBuffer bytes.Buffer
    arrayEncoder := json.NewEncoder(&arrayBuffer)
    arrayEncoder.SetIndent("", "  ")
    
    err := arrayEncoder.Encode(books)
    if err != nil {
        fmt.Printf("編碼陣列錯誤: %v\n", err)
    } else {
        fmt.Printf("書籍陣列編碼結果:\n%s\n", arrayBuffer.String())
    }
}

func main() {
    jsonEncoderExample()
}

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "strings"
)

/* JSON Decoder 使用示例 */
func jsonDecoderExample() {
    fmt.Println("JSON Decoder 示例:")
    
    // 模擬包含多個 JSON 物件的資料流
    jsonStream := `{"id":1,"name":"Alice","age":25}
    {"id":2,"name":"Bob","age":30}
    {"id":3,"name":"Charlie","age":35}
    {"id":4,"name":"Diana","age":28}`
    
    type User struct {
        ID   int    `json:"id"`
        Name string `json:"name"`
        Age  int    `json:"age"`
    }
    
    // 建立 Decoder
    reader := strings.NewReader(jsonStream)
    decoder := json.NewDecoder(reader)
    
    fmt.Println("逐一解碼 JSON 物件:")
    
    var users []User
    for {
        var user User
        err := decoder.Decode(&user)
        
        if err != nil {
            // 檢查是否是檔案結束
            if err.Error() == "EOF" {
                fmt.Println("資料流結束")
                break
            }
            fmt.Printf("解碼錯誤: %v\n", err)
            break
        }
        
        users = append(users, user)
        fmt.Printf("解碼使用者: ID=%d, Name=%s, Age=%d\n", 
            user.ID, user.Name, user.Age)
    }
    
    fmt.Printf("\n總共解碼了 %d 個使用者\n", len(users))
    
    // 使用 UseNumber 選項處理數字
    fmt.Println("\n使用 UseNumber 選項:")
    
    jsonWithNumbers := `{"integer": 123, "float": 456.789, "big_number": 12345678901234567890}`
    
    numberReader := strings.NewReader(jsonWithNumbers)
    numberDecoder := json.NewDecoder(numberReader)
    numberDecoder.UseNumber() // 數字保持為 json.Number 型別
    
    var result map[string]interface{}
    err := numberDecoder.Decode(&result)
    if err != nil {
        fmt.Printf("解碼數字錯誤: %v\n", err)
        return
    }
    
    for key, value := range result {
        fmt.Printf("%s: %v (型別: %T)\n", key, value, value)
        
        // 如果是 json.Number，可以轉換為不同型別
        if num, ok := value.(json.Number); ok {
            if intVal, err := num.Int64(); err == nil {
                fmt.Printf("  轉為 int64: %d\n", intVal)
            }
            if floatVal, err := num.Float64(); err == nil {
                fmt.Printf("  轉為 float64: %f\n", floatVal)
            }
            fmt.Printf("  原始字串: %s\n", num.String())
        }
    }
}

func main() {
    jsonDecoderExample()
}

### 編碼與解碼 JSON 串流

JSON 串流處理是指處理連續的 JSON 物件，而不是單一的 JSON 文件。這在處理大量資料、日誌檔案、或即時資料流時特別有用。

### 串流處理的優勢：
- 記憶體使用效率高
- 可以處理任意大小的資料
- 支援即時處理
- 適合管道處理模式

### 使用場景：
- 日誌檔案處理
- 資料庫匯出/匯入
- 即時資料流處理
- 大型 JSON 陣列處理

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "os"
    "strings"
)

/* JSON 串流處理示例 */
type LogEntry struct {
    Timestamp string `json:"timestamp"`
    Level     string `json:"level"`
    Message   string `json:"message"`
    UserID    int    `json:"user_id,omitempty"`
}

func jsonStreamExample() {
    fmt.Println("JSON 串流處理示例:")
    
    // 模擬日誌資料串流
    logEntries := []LogEntry{
        {Timestamp: "2023-12-25T10:00:00Z", Level: "INFO", Message: "使用者登入", UserID: 1001},
        {Timestamp: "2023-12-25T10:01:00Z", Level: "ERROR", Message: "資料庫連接失敗"},
        {Timestamp: "2023-12-25T10:02:00Z", Level: "WARN", Message: "記憶體使用率過高"},
        {Timestamp: "2023-12-25T10:03:00Z", Level: "INFO", Message: "備份完成"},
        {Timestamp: "2023-12-25T10:04:00Z", Level: "DEBUG", Message: "緩存清理"},
    }
    
    fmt.Println("寫入 JSON 串流到標準輸出:")
    
    // 建立編碼器寫入到標準輸出
    encoder := json.NewEncoder(os.Stdout)
    encoder.SetIndent("", "  ")
    
    // 逐一編碼每個日誌條目
    for i, entry := range logEntries {
        fmt.Printf("--- 日誌條目 %d ---\n", i+1)
        err := encoder.Encode(entry)
        if err != nil {
            fmt.Printf("編碼錯誤: %v\n", err)
        }
    }
}

func main() {
    jsonStreamExample()
}

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "strings"
)

/* 處理 JSON Lines (JSONL) 格式 */
func jsonLinesExample() {
    fmt.Println("JSON Lines (JSONL) 處理示例:")
    
    // JSONL 格式：每行一個 JSON 物件
    jsonlData := `{"name":"Alice","score":95,"subject":"Math"}
{"name":"Bob","score":87,"subject":"Science"}
{"name":"Charlie","score":92,"subject":"English"}
{"name":"Diana","score":98,"subject":"History"}`
    
    type StudentScore struct {
        Name    string `json:"name"`
        Score   int    `json:"score"`
        Subject string `json:"subject"`
    }
    
    // 分割每一行並解碼
    lines := strings.Split(jsonlData, "\n")
    var students []StudentScore
    
    fmt.Println("解析 JSONL 資料:")
    
    for i, line := range lines {
        line = strings.TrimSpace(line)
        if line == "" {
            continue
        }
        
        var student StudentScore
        err := json.Unmarshal([]byte(line), &student)
        if err != nil {
            fmt.Printf("解析第 %d 行錯誤: %v\n", i+1, err)
            continue
        }
        
        students = append(students, student)
        fmt.Printf("第 %d 行: %s - %s: %d 分\n", 
            i+1, student.Name, student.Subject, student.Score)
    }
    
    // 統計資料
    fmt.Printf("\n統計資料:\n")
    fmt.Printf("總學生數: %d\n", len(students))
    
    totalScore := 0
    for _, student := range students {
        totalScore += student.Score
    }
    
    if len(students) > 0 {
        avgScore := float64(totalScore) / float64(len(students))
        fmt.Printf("平均分數: %.2f\n", avgScore)
    }
    
    // 找出最高分
    if len(students) > 0 {
        maxScore := students[0].Score
        maxStudent := students[0].Name
        
        for _, student := range students {
            if student.Score > maxScore {
                maxScore = student.Score
                maxStudent = student.Name
            }
        }
        
        fmt.Printf("最高分: %s (%d 分)\n", maxStudent, maxScore)
    }
}

func main() {
    jsonLinesExample()
}

### 自訂 JSON 解析

有時候預設的 JSON 編碼和解碼行為不能滿足需求，這時可以透過實作特定的介面來自訂 JSON 處理行為。

### 可實作的介面：
- `json.Marshaler`: 自訂 JSON 編碼
- `json.Unmarshaler`: 自訂 JSON 解碼
- `encoding.TextMarshaler`: 自訂文字編碼
- `encoding.TextUnmarshaler`: 自訂文字解碼

### 使用場景：
- 特殊的時間格式處理
- 自訂的資料結構序列化
- 資料驗證和轉換
- 相容性處理

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "strconv"
    "strings"
    "time"
)

/* 自訂時間格式 */
type CustomTime struct {
    time.Time
}

const customTimeFormat = "2006-01-02 15:04:05"

// 實作 json.Marshaler 介面
func (ct CustomTime) MarshalJSON() ([]byte, error) {
    formatted := ct.Time.Format(customTimeFormat)
    return json.Marshal(formatted)
}

// 實作 json.Unmarshaler 介面
func (ct *CustomTime) UnmarshalJSON(data []byte) error {
    var timeStr string
    if err := json.Unmarshal(data, &timeStr); err != nil {
        return err
    }
    
    parsed, err := time.Parse(customTimeFormat, timeStr)
    if err != nil {
        return err
    }
    
    ct.Time = parsed
    return nil
}

/* 自訂版本類型 */
type Version struct {
    Major int
    Minor int
    Patch int
}

// 實作 json.Marshaler 介面 - 編碼為 "major.minor.patch" 格式
func (v Version) MarshalJSON() ([]byte, error) {
    versionStr := fmt.Sprintf("%d.%d.%d", v.Major, v.Minor, v.Patch)
    return json.Marshal(versionStr)
}

// 實作 json.Unmarshaler 介面 - 從 "major.minor.patch" 格式解碼
func (v *Version) UnmarshalJSON(data []byte) error {
    var versionStr string
    if err := json.Unmarshal(data, &versionStr); err != nil {
        return err
    }
    
    parts := strings.Split(versionStr, ".")
    if len(parts) != 3 {
        return fmt.Errorf("無效的版本格式: %s", versionStr)
    }
    
    major, err := strconv.Atoi(parts[0])
    if err != nil {
        return fmt.Errorf("無效的主版本號: %s", parts[0])
    }
    
    minor, err := strconv.Atoi(parts[1])
    if err != nil {
        return fmt.Errorf("無效的次版本號: %s", parts[1])
    }
    
    patch, err := strconv.Atoi(parts[2])
    if err != nil {
        return fmt.Errorf("無效的修訂版本號: %s", parts[2])
    }
    
    v.Major = major
    v.Minor = minor
    v.Patch = patch
    return nil
}

type Application struct {
    Name      string     `json:"name"`
    Version   Version    `json:"version"`
    CreatedAt CustomTime `json:"created_at"`
    UpdatedAt CustomTime `json:"updated_at"`
}

func customJSONExample() {
    fmt.Println("自訂 JSON 解析示例:")
    
    // 建立應用程式實例
    app := Application{
        Name:    "My App",
        Version: Version{Major: 1, Minor: 2, Patch: 3},
        CreatedAt: CustomTime{time.Date(2023, 12, 25, 10, 30, 0, 0, time.UTC)},
        UpdatedAt: CustomTime{time.Date(2023, 12, 25, 15, 45, 30, 0, time.UTC)},
    }
    
    // 編碼為 JSON
    appJSON, err := json.MarshalIndent(app, "", "  ")
    if err != nil {
        fmt.Printf("編碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("編碼結果:\n%s\n\n", appJSON)
    
    // 解碼 JSON
    jsonStr := `{
        "name": "Another App",
        "version": "2.1.0",
        "created_at": "2023-12-20 09:15:30",
        "updated_at": "2023-12-25 16:20:45"
    }`
    
    var newApp Application
    err = json.Unmarshal([]byte(jsonStr), &newApp)
    if err != nil {
        fmt.Printf("解碼錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("解碼結果:\n")
    fmt.Printf("名稱: %s\n", newApp.Name)
    fmt.Printf("版本: %d.%d.%d\n", newApp.Version.Major, newApp.Version.Minor, newApp.Version.Patch)
    fmt.Printf("建立時間: %v\n", newApp.CreatedAt.Time)
    fmt.Printf("更新時間: %v\n", newApp.UpdatedAt.Time)
}

func main() {
    customJSONExample()
}

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "strings"
)

/* 自訂 JSON 驗證和轉換 */
type Email struct {
    Address string
}

// 實作 json.Marshaler 介面
func (e Email) MarshalJSON() ([]byte, error) {
    return json.Marshal(e.Address)
}

// 實作 json.Unmarshaler 介面，包含驗證邏輯
func (e *Email) UnmarshalJSON(data []byte) error {
    var address string
    if err := json.Unmarshal(data, &address); err != nil {
        return err
    }
    
    // 簡單的電子郵件驗證
    if !strings.Contains(address, "@") {
        return fmt.Errorf("無效的電子郵件格式: %s", address)
    }
    
    parts := strings.Split(address, "@")
    if len(parts) != 2 || parts[0] == "" || parts[1] == "" {
        return fmt.Errorf("無效的電子郵件格式: %s", address)
    }
    
    // 轉換為小寫
    e.Address = strings.ToLower(address)
    return nil
}

type ContactInfo struct {
    Name  string `json:"name"`
    Email Email  `json:"email"`
    Phone string `json:"phone,omitempty"`
}

func validationExample() {
    fmt.Println("JSON 驗證和轉換示例:")
    
    // 測試資料
    testCases := []string{
        `{"name":"Alice","email":"ALICE@Example.COM","phone":"+1-555-0123"}`,
        `{"name":"Bob","email":"bob@company.org"}`,
        `{"name":"Charlie","email":"invalid-email"}`, // 這個會失敗
        `{"name":"Diana","email":"@missing-local.com"}`, // 這個也會失敗
    }
    
    for i, testJSON := range testCases {
        fmt.Printf("\n--- 測試案例 %d ---\n", i+1)
        fmt.Printf("輸入: %s\n", testJSON)
        
        var contact ContactInfo
        err := json.Unmarshal([]byte(testJSON), &contact)
        
        if err != nil {
            fmt.Printf("解碼失敗: %v\n", err)
            continue
        }
        
        fmt.Printf("解碼成功:\n")
        fmt.Printf("  名稱: %s\n", contact.Name)
        fmt.Printf("  電子郵件: %s (已正規化)\n", contact.Email.Address)
        if contact.Phone != "" {
            fmt.Printf("  電話: %s\n", contact.Phone)
        }
        
        // 重新編碼以驗證結果
        outputJSON, err := json.MarshalIndent(contact, "", "  ")
        if err != nil {
            fmt.Printf("重新編碼失敗: %v\n", err)
        } else {
            fmt.Printf("重新編碼結果:\n%s\n", outputJSON)
        }
    }
}

func main() {
    validationExample()
}

## net/http

`net/http` 套件是 Go 中處理 HTTP 通訊的核心套件。它提供了建立 HTTP 客戶端和伺服器的完整功能，設計簡潔而功能強大。

### 核心特性：
- 完整的 HTTP/1.1 和 HTTP/2 支援
- 內建的 HTTP 客戶端
- 強大的 HTTP 伺服器
- 中間件支援
- 自動處理 cookies
- TLS/HTTPS 支援
- 請求/回應的完整控制

### 主要組件：
- `http.Client`: HTTP 客戶端
- `http.Server`: HTTP 伺服器
- `http.Handler`: 處理請求的介面
- `http.Request`: HTTP 請求
- `http.Response`: HTTP 回應

### 用戶端

HTTP 客戶端用於發送 HTTP 請求到遠端伺服器。Go 的 HTTP 客戶端支援所有標準的 HTTP 方法，並提供了豐富的配置選項。

### 主要功能：
- 支援所有 HTTP 方法 (GET, POST, PUT, DELETE 等)
- 自動處理重定向
- Cookie 管理
- 超時設定
- 自訂 Transport
- 代理支援
- 身份驗證

### 使用模式：
- 簡單請求：使用 `http.Get`, `http.Post` 等便利函式
- 複雜請求：建立 `http.Request` 並使用 `http.Client`
- 自訂客戶端：建立配置的 `http.Client` 實例

In [ ]:
package main

import (
    "fmt"
    "io"
    "net/http"
)

/* 基本 HTTP GET 請求 */
func basicHTTPGetExample() {
    fmt.Println("基本 HTTP GET 請求示例:")
    
    // 使用便利函式進行 GET 請求
    resp, err := http.Get("https://httpbin.org/get")
    if err != nil {
        fmt.Printf("請求失敗: %v\n", err)
        return
    }
    defer resp.Body.Close() // 重要：總是關閉回應主體
    
    // 檢查狀態碼
    fmt.Printf("狀態碼: %d\n", resp.StatusCode)
    fmt.Printf("狀態: %s\n", resp.Status)
    
    // 顯示回應標頭
    fmt.Println("\n回應標頭:")
    for key, values := range resp.Header {
        for _, value := range values {
            fmt.Printf("  %s: %s\n", key, value)
        }
    }
    
    // 讀取回應主體
    body, err := io.ReadAll(resp.Body)
    if err != nil {
        fmt.Printf("讀取回應主體失敗: %v\n", err)
        return
    }
    
    fmt.Printf("\n回應主體長度: %d 位元組\n", len(body))
    
    // 顯示回應內容的前 200 個字元
    bodyStr := string(body)
    if len(bodyStr) > 200 {
        bodyStr = bodyStr[:200] + "..."
    }
    fmt.Printf("回應內容: %s\n", bodyStr)
}

func main() {
    basicHTTPGetExample()
}

In [ ]:
package main

import (
    "bytes"
    "encoding/json"
    "fmt"
    "io"
    "net/http"
)

/* HTTP POST 請求與 JSON 資料 */
func httpPostJSONExample() {
    fmt.Println("HTTP POST JSON 請求示例:")
    
    // 準備要發送的資料
    requestData := map[string]interface{}{
        "name":    "John Doe",
        "email":   "john@example.com",
        "age":     30,
        "hobbies": []string{"reading", "swimming", "coding"},
    }
    
    // 將資料編碼為 JSON
    jsonData, err := json.Marshal(requestData)
    if err != nil {
        fmt.Printf("JSON 編碼失敗: %v\n", err)
        return
    }
    
    fmt.Printf("發送的 JSON 資料: %s\n", jsonData)
    
    // 發送 POST 請求
    resp, err := http.Post(
        "https://httpbin.org/post", 
        "application/json", 
        bytes.NewBuffer(jsonData),
    )
    if err != nil {
        fmt.Printf("POST 請求失敗: %v\n", err)
        return
    }
    defer resp.Body.Close()
    
    // 檢查回應
    fmt.Printf("\n回應狀態: %s\n", resp.Status)
    fmt.Printf("Content-Type: %s\n", resp.Header.Get("Content-Type"))
    
    // 讀取並解析回應
    body, err := io.ReadAll(resp.Body)
    if err != nil {
        fmt.Printf("讀取回應失敗: %v\n", err)
        return
    }
    
    // 嘗試解析回應為 JSON
    var response map[string]interface{}
    err = json.Unmarshal(body, &response)
    if err != nil {
        fmt.Printf("JSON 解析失敗: %v\n", err)
        fmt.Printf("原始回應: %s\n", string(body)[:200]+"...")
        return
    }
    
    // 顯示伺服器收到的資料
    fmt.Println("\n伺服器收到的資料:")
    if data, ok := response["json"]; ok {
        prettyJSON, _ := json.MarshalIndent(data, "", "  ")
        fmt.Printf("%s\n", prettyJSON)
    }
    
    // 顯示請求標頭資訊
    if headers, ok := response["headers"]; ok {
        fmt.Println("\n請求標頭:")
        if headersMap, ok := headers.(map[string]interface{}); ok {
            for key, value := range headersMap {
                fmt.Printf("  %s: %v\n", key, value)
            }
        }
    }
}

func main() {
    httpPostJSONExample()
}

In [ ]:
package main

import (
    "fmt"
    "io"
    "net/http"
    "time"
)

/* 自訂 HTTP 客戶端配置 */
func customHTTPClientExample() {
    fmt.Println("自訂 HTTP 客戶端示例:")
    
    // 建立自訂的 HTTP 客戶端
    client := &http.Client{
        Timeout: 10 * time.Second, // 設定請求超時
        Transport: &http.Transport{
            MaxIdleConns:        10,               // 最大閒置連線數
            MaxIdleConnsPerHost: 2,                // 每個主機最大閒置連線數
            IdleConnTimeout:     30 * time.Second, // 閒置連線超時
        },
    }
    
    // 建立 HTTP 請求
    req, err := http.NewRequest("GET", "https://httpbin.org/delay/2", nil)
    if err != nil {
        fmt.Printf("建立請求失敗: %v\n", err)
        return
    }
    
    // 設定自訂標頭
    req.Header.Set("User-Agent", "Go-HTTP-Client/1.0")
    req.Header.Set("Accept", "application/json")
    req.Header.Set("X-Custom-Header", "Custom-Value")
    
    fmt.Println("發送請求 (伺服器會延遲 2 秒回應)...")
    start := time.Now()
    
    // 使用自訂客戶端發送請求
    resp, err := client.Do(req)
    if err != nil {
        fmt.Printf("請求失敗: %v\n", err)
        return
    }
    defer resp.Body.Close()
    
    elapsed := time.Since(start)
    
    fmt.Printf("請求完成，耗時: %v\n", elapsed)
    fmt.Printf("狀態碼: %d\n", resp.StatusCode)
    
    // 顯示一些重要的回應標頭
    fmt.Println("\n重要的回應標頭:")
    headers := []string{"Content-Type", "Content-Length", "Server", "Date"}
    for _, header := range headers {
        if value := resp.Header.Get(header); value != "" {
            fmt.Printf("  %s: %s\n", header, value)
        }
    }
    
    // 讀取回應主體的前一部分
    body, err := io.ReadAll(resp.Body)
    if err != nil {
        fmt.Printf("讀取回應失敗: %v\n", err)
        return
    }
    
    fmt.Printf("\n回應大小: %d 位元組\n", len(body))
    
    // 顯示回應的一部分
    bodyStr := string(body)
    if len(bodyStr) > 300 {
        bodyStr = bodyStr[:300] + "..."
    }
    fmt.Printf("回應內容: %s\n", bodyStr)
}

func main() {
    customHTTPClientExample()
}

### 伺服器

Go 的 HTTP 伺服器功能強大且易於使用。它支援並行處理請求，提供了靈活的路由和中間件機制。

### 核心概念：
- `http.Handler` 介面：定義請求處理器
- `http.HandlerFunc`：函式型別的處理器
- `http.ServeMux`：HTTP 請求多工器（路由器）
- `http.Server`：HTTP 伺服器配置

### 主要特性：
- 自動並行處理請求
- 靈活的路由系統
- 中間件支援
- 靜態檔案服務
- TLS/HTTPS 支援
- 優雅關閉
- 請求超時控制

In [ ]:
package main

import (
    "encoding/json"
    "fmt"
    "log"
    "net/http"
    "time"
)

/* 基本 HTTP 伺服器示例 */

// 根路徑處理器
func homeHandler(w http.ResponseWriter, r *http.Request) {
    fmt.Fprintf(w, "歡迎來到 Go HTTP 伺服器！\n")
    fmt.Fprintf(w, "當前時間: %s\n", time.Now().Format("2006-01-02 15:04:05"))
    fmt.Fprintf(w, "請求方法: %s\n", r.Method)
    fmt.Fprintf(w, "請求路徑: %s\n", r.URL.Path)
}

// 處理 JSON 回應
func jsonHandler(w http.ResponseWriter, r *http.Request) {
    // 設定回應標頭
    w.Header().Set("Content-Type", "application/json")
    
    response := map[string]interface{}{
        "message":   "Hello, JSON!",
        "timestamp": time.Now().Unix(),
        "method":    r.Method,
        "path":      r.URL.Path,
        "headers":   r.Header,
    }
    
    // 編碼為 JSON 並發送回應
    json.NewEncoder(w).Encode(response)
}

// 處理查詢參數
func queryHandler(w http.ResponseWriter, r *http.Request) {
    w.Header().Set("Content-Type", "application/json")
    
    // 解析查詢參數
    query := r.URL.Query()
    
    response := map[string]interface{}{
        "message": "查詢參數處理",
        "params":  make(map[string]interface{}),
    }
    
    params := response["params"].(map[string]interface{})
    
    // 處理每個查詢參數
    for key, values := range query {
        if len(values) == 1 {
            params[key] = values[0]
        } else {
            params[key] = values
        }
    }
    
    json.NewEncoder(w).Encode(response)
}

// 處理 POST 請求
func postHandler(w http.ResponseWriter, r *http.Request) {
    if r.Method != http.MethodPost {
        http.Error(w, "只支援 POST 方法", http.StatusMethodNotAllowed)
        return
    }
    
    w.Header().Set("Content-Type", "application/json")
    
    // 讀取請求主體
    var requestData map[string]interface{}
    err := json.NewDecoder(r.Body).Decode(&requestData)
    if err != nil {
        http.Error(w, "無法解析 JSON: "+err.Error(), http.StatusBadRequest)
        return
    }
    
    response := map[string]interface{}{
        "message":      "POST 請求處理成功",
        "received_data": requestData,
        "timestamp":     time.Now().Unix(),
    }
    
    json.NewEncoder(w).Encode(response)
}

func basicHTTPServerExample() {
    fmt.Println("基本 HTTP 伺服器示例:")
    
    // 建立路由
    mux := http.NewServeMux()
    
    // 註冊處理器
    mux.HandleFunc("/", homeHandler)
    mux.HandleFunc("/json", jsonHandler)
    mux.HandleFunc("/query", queryHandler)
    mux.HandleFunc("/post", postHandler)
    
    // 建立伺服器
    server := &http.Server{
        Addr:         ":8080",
        Handler:      mux,
        ReadTimeout:  10 * time.Second,
        WriteTimeout: 10 * time.Second,
    }
    
    fmt.Println("伺服器啟動在 http://localhost:8080")
    fmt.Println("可用的端點:")
    fmt.Println("  / - 首頁")
    fmt.Println("  /json - JSON 回應")
    fmt.Println("  /query?name=value - 查詢參數處理")
    fmt.Println("  /post - POST 請求處理 (需要 JSON 主體)")
    
    // 啟動伺服器
    log.Fatal(server.ListenAndServe())
}

func main() {
    basicHTTPServerExample()
}

In [ ]:
package main

import (
    "context"
    "encoding/json"
    "fmt"
    "log"
    "net/http"
    "os"
    "os/signal"
    "sync"
    "syscall"
    "time"
)

/* 進階 HTTP 伺服器：中間件和優雅關閉 */

// 記錄中間件
func loggingMiddleware(next http.Handler) http.Handler {
    return http.HandlerFunc(func(w http.ResponseWriter, r *http.Request) {
        start := time.Now()
        
        // 呼叫下一個處理器
        next.ServeHTTP(w, r)
        
        // 記錄請求資訊
        duration := time.Since(start)
        log.Printf("%s %s %v", r.Method, r.URL.Path, duration)
    })
}

// CORS 中間件
func corsMiddleware(next http.Handler) http.Handler {
    return http.HandlerFunc(func(w http.ResponseWriter, r *http.Request) {
        // 設定 CORS 標頭
        w.Header().Set("Access-Control-Allow-Origin", "*")
        w.Header().Set("Access-Control-Allow-Methods", "GET, POST, PUT, DELETE, OPTIONS")
        w.Header().Set("Access-Control-Allow-Headers", "Content-Type, Authorization")
        
        // 處理預檢請求
        if r.Method == http.MethodOptions {
            w.WriteHeader(http.StatusOK)
            return
        }
        
        next.ServeHTTP(w, r)
    })
}

// 簡單的記憶體儲存
type MemoryStore struct {
    mu   sync.RWMutex
    data map[string]interface{}
}

func NewMemoryStore() *MemoryStore {
    return &MemoryStore{
        data: make(map[string]interface{}),
    }
}

func (ms *MemoryStore) Set(key string, value interface{}) {
    ms.mu.Lock()
    defer ms.mu.Unlock()
    ms.data[key] = value
}

func (ms *MemoryStore) Get(key string) (interface{}, bool) {
    ms.mu.RLock()
    defer ms.mu.RUnlock()
    value, exists := ms.data[key]
    return value, exists
}

func (ms *MemoryStore) Delete(key string) {
    ms.mu.Lock()
    defer ms.mu.Unlock()
    delete(ms.data, key)
}

func (ms *MemoryStore) GetAll() map[string]interface{} {
    ms.mu.RLock()
    defer ms.mu.RUnlock()
    
    result := make(map[string]interface{})
    for k, v := range ms.data {
        result[k] = v
    }
    return result
}

// API 處理器
type APIServer struct {
    store *MemoryStore
}

func (api *APIServer) handleItems(w http.ResponseWriter, r *http.Request) {
    w.Header().Set("Content-Type", "application/json")
    
    switch r.Method {
    case http.MethodGet:
        // 獲取所有項目
        items := api.store.GetAll()
        json.NewEncoder(w).Encode(map[string]interface{}{
            "items": items,
            "count": len(items),
        })
        
    case http.MethodPost:
        // 創建新項目
        var item map[string]interface{}
        if err := json.NewDecoder(r.Body).Decode(&item); err != nil {
            http.Error(w, "無效的 JSON", http.StatusBadRequest)
            return
        }
        
        key := fmt.Sprintf("item_%d", time.Now().UnixNano())
        item["id"] = key
        item["created_at"] = time.Now().Format(time.RFC3339)
        
        api.store.Set(key, item)
        
        w.WriteHeader(http.StatusCreated)
        json.NewEncoder(w).Encode(item)
        
    default:
        http.Error(w, "方法不被允許", http.StatusMethodNotAllowed)
    }
}

func advancedHTTPServerExample() {
    fmt.Println("進階 HTTP 伺服器示例:")
    
    // 建立 API 伺服器
    apiServer := &APIServer{
        store: NewMemoryStore(),
    }
    
    // 建立路由
    mux := http.NewServeMux()
    mux.HandleFunc("/api/items", apiServer.handleItems)
    mux.HandleFunc("/health", func(w http.ResponseWriter, r *http.Request) {
        json.NewEncoder(w).Encode(map[string]string{
            "status": "OK",
            "time":   time.Now().Format(time.RFC3339),
        })
    })
    
    // 應用中間件
    handler := loggingMiddleware(corsMiddleware(mux))
    
    // 建立伺服器
    server := &http.Server{
        Addr:         ":8080",
        Handler:      handler,
        ReadTimeout:  15 * time.Second,
        WriteTimeout: 15 * time.Second,
    }
    
    // 優雅關閉
    go func() {
        c := make(chan os.Signal, 1)
        signal.Notify(c, os.Interrupt, syscall.SIGTERM)
        
        <-c
        fmt.Println("\n正在優雅關閉伺服器...")
        
        ctx, cancel := context.WithTimeout(context.Background(), 15*time.Second)
        defer cancel()
        
        if err := server.Shutdown(ctx); err != nil {
            log.Printf("伺服器關閉錯誤: %v", err)
        }
    }()
    
    fmt.Println("伺服器啟動在 http://localhost:8080")
    fmt.Println("可用的端點:")
    fmt.Println("  GET /api/items - 獲取所有項目")
    fmt.Println("  POST /api/items - 創建新項目")
    fmt.Println("  GET /health - 健康檢查")
    fmt.Println("\n按 Ctrl+C 停止伺服器")
    
    // 啟動伺服器
    if err := server.ListenAndServe(); err != http.ErrServerClosed {
        log.Fatalf("伺服器啟動失敗: %v", err)
    }
    
    fmt.Println("伺服器已停止")
}

func main() {
    advancedHTTPServerExample()
}

## 總結

Go 的標準程式庫提供了構建現代應用程式所需的強大工具。透過本章的學習，我們深入了解了四個關鍵套件：

### 核心收穫：

**1. io 與朋友**
- 理解了 Go 的 I/O 抽象和介面設計
- 掌握了 Reader、Writer、Closer 等核心介面
- 學會了使用 bufio 進行高效的緩衝 I/O 操作
- 了解了 I/O 工具函式的強大功能

**2. time 套件**
- 掌握了時間的創建、格式化和解析
- 理解了單調時間的概念和重要性
- 學會了使用 Timer 和 Ticker 進行時間控制
- 了解了時區處理和時間計算

**3. encoding/json**
- 熟練掌握了 JSON 的編碼和解碼
- 學會了使用 struct tags 控制 JSON 行為
- 了解了 Encoder/Decoder 的串流處理能力
- 掌握了自訂 JSON 處理的高級技巧

**4. net/http**
- 學會了建立強大的 HTTP 客戶端
- 掌握了 HTTP 伺服器的開發技巧
- 了解了中間件模式和優雅關閉
- 具備了構建 RESTful API 的能力

### 設計哲學體現：

- **介面導向**：所有套件都大量使用介面，提供了優秀的組合性
- **簡潔性**：API 設計簡單直觀，易於學習和使用
- **一致性**：不同套件之間遵循相似的設計模式
- **效能**：內建了許多效能最佳化，如連線池、緩衝等

### 實際應用：

這些標準程式庫套件是構建以下類型應用的基礎：
- Web 服務和 REST API
- 微服務架構
- 資料處理管道
- 網路工具和爬蟲
- 即時應用程式

掌握了這些標準程式庫，你就具備了使用 Go 開發各種實際應用程式的堅實基礎。